In [3]:
import numpy as np
from scipy.stats import multivariate_normal

Log:

- 11/10 Fixed bugs: H^(-1/2) formula, shapes of theta & epsilons
- 11/10 Updated codes to reduce sampling time

# Functions to be tested

For now, use a very simple function - quadratic with max value 0, and theta* = [0.5, 0.5] 

Also note that we will maximize the function, not minimize.

In [67]:
def F(theta):
    if theta.ndim == 1:
        theta = np.expand_dims(theta, 0)
    return -np.sum((theta - 0.5) ** 2, axis=tuple(range(theta.ndim)[1:]))

# Gradient benchmark

This is exactly the Algorithm 1 in the RL paper.

In [68]:
def ES_benchmark_gradient(alpha, sigma, theta_0, num_samples, time_steps):
    d = theta_0.shape[0]
    theta_t = theta_0
    n = num_samples
    for t in range(time_steps):
        #**** sample epsilons ****#
        epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n) # n by d
        #**** compute function values ****#
        F_list = []
        for i in range(n):
            F_val = F(theta_t + sigma * epsilons[i])
            F_list.append(F_val)
        #**** update theta ****#
        new_theta = theta_t
        for i in range(n):
            new_theta += alpha / (n*sigma) * F_list[i] * epsilons[i] 
        theta_t = new_theta
    return theta_t, F(theta_t)

In [69]:
ES_benchmark_gradient(alpha=1e-3, sigma=0.1, theta_0 = np.array([1.0,1.0]), num_samples = 50, time_steps = 5000)

(array([0.49979673, 0.50081561]), array([-7.06536921e-07]))

# HessAware Benchmark

This implements the Hess Aware algorithm in Zhang's paper. 

The Hessian estimate comes from Section 4.1.

In [70]:
def ES_benchmark_hess_aware(alpha, sigma, theta_0, num_samples, time_steps, p, H_lambda):
    d = theta_0.shape[0]
    theta_t = theta_0.reshape((d,1))
    n = num_samples
    H = None
    for t in range(time_steps):
        #**** sample epsilons ****#
        epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n) # n by d
        #**** compute function values ****#
        F_plus_list = []; F_minus_list = []; F_list = []
        for i in range(n):
            F_plus_list.append(F(theta_t + sigma * epsilons[i].reshape((d,1))))
            F_minus_list.append(F(theta_t - sigma * epsilons[i].reshape((d,1))))
            F_list.append(F(theta_t))
        #**** compute Hessian every p steps ****#
        if t % p == 0:
            H = np.zeros((d,d))
            for i in range(n):
                e_i = epsilons[i].reshape((d,1))
                e_i_trans = np.transpose(e_i)
                H += (F_plus_list[i] + F_minus_list[i] - 2*F_list[i]) * (e_i @ e_i_trans)
            H /= 2*(sigma**2)*n
            H += H_lambda * np.identity(d)
        #**** update theta: compute g ****#
        u, s, vh = np.linalg.svd(H)
        H_nh = u @ np.diag(s**-0.5) @ vh
        g = 0
        for i in range(n):
            e_i = epsilons[i].reshape((d,1))
            F_new = F(theta_t +  sigma* (H_nh @ e_i)  )
            g += ((F_new - F(theta_t)) / sigma ) * (H_nh @ e_i) / n
        #**** update theta: the rest ****#
        new_theta = theta_t + alpha * g
        theta_t = new_theta
        
    return theta_t, F(theta_t), H

In [71]:
ES_benchmark_hess_aware(alpha=0.1, sigma=0.01, theta_0=np.array([1.0,1.0]), num_samples = 1000, time_steps = 100, p = 10, H_lambda = 0)

(array([[0.05275927, 0.96918208],
        [0.96885678, 0.05579611]]),
 array([-0.42015609, -0.41714377]),
 array([[-6.05413759, -0.22772953],
        [-0.49210759, -4.71143407]]))

# First Hessian-based method

We use the Hessian estimate as in the write-up document on Overleaf.

Then, the same Newton's method as in Zhang's paper is used to update theta with the Hessian estimate.

As before, alpha is the learning rate. The parameter p defines how often we re-compute the Hessian.

In [240]:
delta_g_lst = []
diag_lst = []
def ES_hessian(alpha, sigma, theta_0, num_samples, time_steps, p, H_lambda):
    d = theta_0.shape[0]
    theta_t = theta_0
    n = num_samples
    H = None
    for t in range(time_steps):
        #**** sample epsilons ****#
        epsilons = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d), size = n) # n by d
        #**** compute Hessian every p steps ****#
        if t % p == 0:
            Fs = F(theta_t + sigma*epsilons)
            eps = np.expand_dims(epsilons, -1)
            H_samples = ((eps*np.transpose(eps, (0, 2, 1)) - np.identity(d))* Fs.reshape(-1, 1, 1))/(sigma**2)
            H = H_samples.mean(axis=0)
        #**** update theta: compute g ****#
        u, s, vh = np.linalg.svd(H)
        H_nh = u @ np.diag(s**-0.5) @ vh
        H_nh_3d = np.ones((n, d, d)) * H_nh
        Fs = F(theta_t +  sigma* np.transpose((H_nh @ np.transpose(epsilons) )) ) - F(theta_t)
        eps = np.expand_dims(epsilons, -1)
        g_samples =  H_nh_3d @ eps * Fs.reshape(-1, 1, 1)/sigma
        g = g_samples.mean(axis=0).ravel()
        
        #**** update theta: the rest ****#
        new_theta = theta_t + alpha * g
        theta_t = new_theta
        
    return theta_t, F(theta_t), H

In [252]:
res = ES_hessian(alpha=0.5, sigma=0.5, theta_0=np.array([1.0,1.0]), num_samples = 1000, time_steps = 1000, p = 10, H_lambda = 0)
print(res)

(array([0.48280216, 0.4788555 ]), array([-0.00074286]), array([[-2.42753549, -0.1397509 ],
       [-0.1397509 , -2.20456166]]))
